In [1]:
%pylab inline
import gzip
import pickle
import glob
import os
import sys

Populating the interactive namespace from numpy and matplotlib


In [2]:
MyGraph = {}

IDir = "../../../strong_ties/data/twitter/twitter/data-01-09_company_like/"
outFile = "TWT_NET.pkl"

zipped_f = False
max_loaded_events=100000000

caller_idx = 0
called_idx = 1
clr_company_idx = 2
cld_company_idx = 3

In [3]:
lineFetcher = lambda vs: (vs[caller_idx], vs[called_idx],\
                                True, True, -1)

In [4]:
ApriFile = gzip.open if zipped_f else open # Gzipped input?

In [5]:
fnames = sorted([f for f in sorted(os.listdir(IDir)) if f[0] != "."])

In [6]:
EV_TOT = 0
for file_index, fn in enumerate(fnames):
    sys.stdout.write("\rFirst round, file %03d of %03d..." % (file_index+1, len(fnames)) )
    sys.stdout.flush()
    sliceNum = 0
    readDone = False
    while not readDone:
        Listone = []
        with ApriFile(os.path.join(IDir, fn), 'rb') as of:
            for _ in range(max_loaded_events*sliceNum): next(of)

            for ln in of:
                lineValues = ln.strip().split()
                Listone.append(lineFetcher(lineValues))

                if len(Listone) >= max_loaded_events:
                    sliceNum += 1
                    break
            else:
                readDone = True
        for clr, cld, comp_clr, comp_cld, time in Listone:
            if clr != cld:
                EV_TOT += 1
                #time_fin = time if time_scheme == "given" else EV_TOT
                if clr not in MyGraph:
                    MyGraph[clr] = {cld: 1}
                else:
                    try:
                        MyGraph[clr][cld] += 1
                    except KeyError:
                        MyGraph[clr][cld] = 1

                if cld not in MyGraph:
                    MyGraph[cld] = {clr: 0}
                else:
                    if clr not in MyGraph[cld]:
                        MyGraph[cld][clr] = 0
        del Listone

First round, file 273 of 273...

In [7]:
MyGraph[list(MyGraph.keys())[0]]

{b'1': 75,
 b'1543': 20,
 b'8126': 12,
 b'257': 46,
 b'146': 43,
 b'1555': 1,
 b'21698': 8,
 b'4323': 19,
 b'58769': 45,
 b'43870': 5,
 b'274': 6,
 b'141475': 36,
 b'59849': 14,
 b'467307': 17,
 b'491335': 11,
 b'2064': 1}

In [8]:
pickle.dump(MyGraph, gzip.open(outFile, "wb"))